<a href="https://colab.research.google.com/github/Jarred-ZA/FPAI_Wind/blob/main/Feature_Engineering_Research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Feature Engineering - AI for wind energy**

*This is a working document to allow for the extraction, manipulation and interogation of the provided datasets*



In [ ]:
!pip install wget
!pip install zenodo-get
!pip install lvm-read
from google.colab import drive
drive.flush_and_unmount()

Drive not mounted, so nothing to flush and unmount.


In [ ]:
#Import relvant libiarys

import numpy as np
import pandas as pd
from scipy.fft import fft, ifft, fftfreq
import random
import matplotlib.pyplot as plt
from scipy.io import loadmat 
import os
import zenodo_get
import lvm_read
import io
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

In [ ]:
# Clone dataset A repo
!git clone https://github.com/ETH-WindMil/Sonkyo-Benchmark

fatal: destination path 'Sonkyo-Benchmark' already exists and is not an empty directory.


In [ ]:
!zenodo_get -r 3229743 -w urls.txt

AttributeError: ignored

In [32]:
with open("urls.txt", "r") as f:
    urls = f.read().splitlines()
    f.close()

#Download 1 folder
urls_np = np.array(urls)
get_url = "\'{}\'".format(urls_np[0])
url_filename = "{}".format(urls_np[0])
filename = url_filename.split('/')
filename = filename[len(filename) - 1]        #xxx.zip
filename_truncated = filename.split('.')[0]   #xxx
case = filename[5]


Case_A_(+00)
A


In [ ]:
#Select only one file to download
!wget -nc $get_url
zf = ZipFile(filename)
for file in zf.namelist():
    # read lvm files 
    if (file.endswith('lvm')):
        with zf.open(file, 'r') as lvm:
            buf=io.BytesIO()
            for line in lvm:
                buf.write(line)
                if line[0:3]=='***':
                    #Everything before was a header, make a new buffer
                    buf.close()
                    buf=io.BytesIO()
        buf.seek(0)
        df = pd.read_csv(buf,sep='\t', skiprows=22) 
        df['file_name'] = file
        df.to_csv(f"/content/{filename_truncated}.csv", mode='a', header=not(os.path.exists(f"/content/{filename_truncated}.csv")), index=False)

    # read description files 
    if (file.endswith('txt')):
        with zf.open(file, 'r') as txt:
            buf=io.BytesIO()
            for line in txt:
                buf.write(line)
            buf.seek(0)
            df = pd.read_csv(buf, header=None, names = ['data'])
            df['var'] = df['data'].str.replace('\t', '').str.split(':').str[0]
            df['val'] = df['data'].str.replace('\t', '').str.split(':').str[1]
            df.pop('data')
            df = df.set_index('var').transpose().reset_index()
            df['file_name'] = file
            df.append (df)                

File ‘Case_A_(+00).zip’ already there; not retrieving.



In [ ]:
!ls

In [ ]:
df_a = pd.read_csv("Case_A.csv", nrows=100)
df_a.head()

In [ ]:
        # read description files 
        if (file.endswith('txt')):
            with zipfile.open(file, 'r') as txt:
                buf=io.BytesIO()
                for line in txt:
                    buf.write(line)
                buf.seek(0)
                df = pd.read_csv(buf, header=None, names = ['data'])
                df['var'] = df['data'].str.replace('\t', '').str.split(':').str[0]
                df['val'] = df['data'].str.replace('\t', '').str.split(':').str[1]
                df.pop('data')
                df = df.set_index('var').transpose().reset_index()
                df['file_name'] = file
                dfs.append (df)

In [ ]:
with open("urls.txt", "r") as f:
    urls = f.read().splitlines()
    f.close()

#Download 1 folder
urls_np = np.array(urls)
get_url = "\'{}\'".format(urls_np[0])
if not(os.path.isdir('Data_Folder')):
    !wget -nc $get_url -O Data_Folder.zip
    !unzip 'Data_Folder.zip' -d Data_Folder
    !rm 'Data_Folder.zip'

filename = '/content/Data_Folder/Case_A_(+00)/Case_A_(+00)_1/sine_sweep.lvm'
lvm = lvm_read.read(filename)

In [ ]:
Channels = lvm[0]['Channel names']
print(Channels.index('Ch2'))
i = Channels.index('Ch2') 
plt.plot(lvm[0]['data'][:,i]);

In [ ]:
# Some dummy data
time = np.linspace(0,np.pi*10,1000)
noise = [random.uniform(-2,2) for i in range(len(time))]
noisy_signal =  np.sin(time) + np.cos(time+np.pi*time) + noise
clean_signal = np.sin(time) + np.cos(time+np.pi*time)

# Apply FFT
fft_coefficients = fft(noisy_signal) # fourier transform
fft_coefficients

# plot orignal signal and inverse fourier transform, shows you can transform signal to frequency domain, then back to time domain
fiax = plt.subplots(figsize=(15,6))
ax.plot(time,noisy_signal,'-',label='Nosiy signal')
ax.plot(time,clean_signal,'-',label='Clean signal')
ax.set_xlabel('Timesteps')
ax.legend()
ax.tick_params(rotation=30,labelsize=15)
plt.tight_layout()
plt.title('Clean vs Noisy signal')

# plot amplitude vs frequency 
n = len(signal)

# get frequencies and psd
freqs = fftfreq(noisy_signal.shape[0]) # x axis of amplitude vs frequency graphs
psd = np.abs(fft_coefficients)/n # psd is amplitude/N, psd or power spectrum density is the magnitude of the coefficients resulting from fourier transform

# plot psd
fig,ax = plt.subplots(figsize=(15,6))
ax.plot(freqs[1:int(n/2)],psd[1:int(n/2)])
ax.set_ylabel('Power spectrum',fontsize=15)
ax.set_xlabel('Frequencies',fontsize=15)
ax.set_title('FFT')
ax.tick_params(labelsize=15)
plt.tight_layout()
plt.title('FFT results')

# lets filter out some signal

# Threshold coefficients to denoise signal
psd_indices = psd > 0.15 # mask
fft_filtered = fft_coefficients*psd_indices

# inverse transform filter coefficients
inverse_transform_filtered = ifft(fft_filtered)

# plot this
fig,ax = plt.subplots(figsize=(15,6))
ax.plot(time,inverse_transform_filtered,'-',label='Inverse fourier filtered')
ax.legend()
ax.set_title('Threshold = 0.01')
ax.set_xlabel('Timesteps',fontsize=15)
ax.tick_params(rotation=30,labelsize=15)
plt.tight_layout()
plt.title('Denoised signal')